In [1]:
import os
from glob import glob

import pandas as pd
import numpy as np
from collections import Counter

import pickle
import gc

from tqdm import tqdm
tqdm.pandas()

C:\Users\_v010ch_\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, make_scorer

In [3]:
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

import catboost as cb
from catboost import CatBoostClassifier

In [4]:
DATA = './data'
DATA_OWN = './data_own'
CLICKSTREAM = 'alfabattle2_abattle_clickstream'
MODELS = './models'
UTILS = './utils'
SUBM = './submissions'

# load data / submit / features

In [5]:
using_features = pickle.load(open(os.path.join(DATA_OWN, 'using_features.pkl'), 'rb'))

In [7]:
#data = pd.read_csv(os.path.join(DATA_OWN, 'data_pred.csv'), parse_dates=['timestamp'])
data = pd.read_csv(os.path.join(DATA_OWN, 'data_pred.csv'), usecols = using_features)
data.head()

,client_freq_main_screen,client_freq_statement,client_freq_credit_info,client_freq_own_transfer,client_freq_mobile_recharge,client_freq_phone_money_transfer,client_freq_card2card_transfer,client_freq_chat,client_freq_card_recharge,client_freq_invest,...,tod_diff_glob_freq_main_screen,tod_diff_glob_freq_statement,tod_diff_glob_freq_credit_info,tod_diff_glob_freq_own_transfer,tod_diff_glob_freq_mobile_recharge,tod_diff_glob_freq_phone_money_transfer,tod_diff_glob_freq_card2card_transfer,tod_diff_glob_freq_chat,tod_diff_glob_freq_card_recharge,tod_diff_glob_freq_invest
0,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.0,...,0.002702,0.004453,-0.022015,0.011315,-0.018217,0.011603,0.006227,0.002242,0.00332,-0.001628
1,0.193548,0.580645,0.032258,0.032258,0.064516,0.064516,0.000000,0.032258,0.0,0.0,...,0.002702,0.004453,-0.022015,0.011315,-0.018217,0.011603,0.006227,0.002242,0.00332,-0.001628
2,0.181818,0.363636,0.272727,0.090909,0.000000,0.000000,0.000000,0.090909,0.0,0.0,...,0.002702,0.004453,-0.022015,0.011315,-0.018217,0.011603,0.006227,0.002242,0.00332,-0.001628
3,0.000000,0.777778,0.111111,0.000000,0.000000,0.000000,0.111111,0.000000,0.0,0.0,...,0.002702,0.004453,-0.022015,0.011315,-0.018217,0.011603,0.006227,0.002242,0.00332,-0.001628
4,0.395833,0.187500,0.000000,0.000000,0.312500,0.000000,0.000000,0.104167,0.0,0.0,...,0.002702,0.004453,-0.022015,0.011315,-0.018217,0.011603,0.006227,0.002242,0.00332,-0.001628


In [8]:
if data.isnull().values.any() == True:
    print('I have a Bad news for you!')
    data.fillna(0, inplace = True)

I have a Bad news for you!


In [9]:
subm = pd.read_csv(os.path.join(DATA, 'alfabattle2_abattle_sample_prediction.csv'))
subm.head()

,client_pin,prediction
0,f0c674b2bb4dc64be607029271d706ec,credit_info
1,90725b54ce77576883813d87749df6bd,credit_info
2,eb0f82d74c7b7bd5eafbd5b5f8cb3e2a,own_transfer
3,831bf4c0ecccc55e536b8cfb9153d672,statement
4,3f1a5a1c492ce877af833113e59c5797,invest


# load models

In [10]:
clf_sgd = pickle.load(open(os.path.join(MODELS, 'clf_sgd.pkl'), 'rb'))
#clf_mlp = pickle.load(os.paht.join(MODELS, 'clf_mlp.pkl'))
clf_lr = pickle.load(open(os.path.join(MODELS, 'clf_lr.pkl'), 'rb'))
clf_lrsvc = pickle.load(open(os.path.join(MODELS, 'clf_lrsvc.pkl'), 'rb'))
#clf_svc = pickle.load(os.paht.join(MODELS, 'clf_svc.pkl'))

#clf_cb  = cb.load_model(os.paht.join(MODELS, 'clf_cb.cbm'), format='cbm')

# make predictions

In [11]:
pred_sgd = clf_sgd.predict(data[using_features])

In [12]:
#pred_mlp = clf_mlp.predict(data[using_features])

In [13]:
#pred_svc = clf_svc.predict(data[using_features])

In [14]:
pred_lr = clf_lr.predict(data[using_features])

In [15]:
pred_lrsvc = clf_lrsvc.predict(data[using_features])

In [16]:
#pred_cb = clf_cb.predict(data[using_features])

In [22]:
%%time
pred_stack = [''] * pred_sgd.shape[0]
for idx in range(pred_sgd.shape[0]):
    cnt = Counter([
                   pred_sgd[idx], 
                   #pred_mlp[idx], 
                   pred_lr[idx], 
                   pred_lrsvc[idx]
               ]).most_common()
    pred_stack[idx] = cnt[0][0]
    
    #if cnt[0][0] == cnt[1][0]:
    #    print('terrible!')

Wall time: 304 ms


# make submit

In [23]:
subm.prediction = pred_stack

In [26]:
subm.to_csv(os.path.join(SUBM, 'subm_client_6diff_3stack.csv'), index = False)